In [1]:
import torch
import numpy as np
import torch.nn as nn
from matplotlib import cm
import matplotlib.pyplot as plt
import torch.utils.data as Data
from torch.autograd import Variable
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split

In [2]:
#loading data
my_matrix = np.loadtxt(open("S4AfterNor.csv"),delimiter=",",skiprows=0)
#using all data for training AE
X, y = my_matrix[:,:-1],my_matrix[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=None)
torch_x = torch.from_numpy(X)
torch_y = torch.from_numpy(y)
test_x = torch.from_numpy(X_test)
test_y = torch.from_numpy(y_test)
y_train = torch.tensor(torch_y, dtype=torch.long)
y_test = torch.tensor(test_y,dtype = torch.long)
x_test = torch.tensor(test_x , dtype=torch.float32)
x_train = torch.tensor(torch_x, dtype=torch.float32)

d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.
d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
d:\softwares\python37\lib\site-packages\ipykernel_launcher.py:12: UserWarn

In [3]:
#parameters
LR = 0.0001
EPOCH = 2000
TIME_STEP = 20
INPUT_SIZE = 60
BATCH_SIZE = 100
train_data = Data.TensorDataset(x_train,y_train)
train_loader = Data.DataLoader(dataset= train_data, batch_size=BATCH_SIZE, shuffle=True,num_workers=0)


In [4]:
#model design
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(382, 400),
            nn.LeakyReLU(),
            nn.Linear(400,300),
            nn.LeakyReLU(),
            nn.Linear(300,200),
            nn.LeakyReLU(),
            nn.Linear(200,100),
            nn.LeakyReLU(),
            nn.Linear(100,50),
            nn.LeakyReLU(),
            nn.Linear(50,3), 
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 50),
            nn.LeakyReLU(),
            nn.Linear(50,100),
            nn.LeakyReLU(),
            nn.Linear(100, 200),
            nn.LeakyReLU(),
            nn.Linear(200,300),
            nn.LeakyReLU(),
            nn.Linear(300,400),
            nn.LeakyReLU(),
            nn.Linear(400, 382),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [5]:
autoencoder =  AutoEncoder()
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(autoencoder.parameters(), lr = LR)

In [ ]:
#training AE
times = 1
Times = []
Loss_encoder = []
for epoch in range(EPOCH):
    for step, (x, y) in enumerate(train_loader):
        b_x = Variable(x)   
        b_y = Variable(x)   
        b_label = Variable(y)               
        encoded, decoded = autoencoder(b_x)

        loss = loss_func(decoded, b_y)     
        optimizer.zero_grad()              
        loss.backward()                    
        optimizer.step()                   
        Loss_encoder.append(loss.data)
        Times.append(times)
        times += 1

        if step % 100 == 0 :
            test_encoded, test_decoded = autoencoder(x_test)
            test_loss = loss_func(test_decoded,x_test)
            


In [ ]:
#loading generated data
Non_MI_data = np.load('Non_MI_data.npy')
Non_MI_label = np.load('Non_MI_label.npy')

MI_generated_data = np.load('MI_generated_data_4500.npy')
MI_Non_MI_label = np.load('MI_generatedl_Non_MI.npy')
MI_data = np.load('MI_data.npy')
MI_label = np.load('MI_label.npy')

my_matrix = np.vstack((MI_generated_data[:57],MI_data))

y = []
for i in MI_Non_MI_label[:57]:
    y.append(1)   
for i in MI_label:
    y.append(0)
    
X = my_matrix
y = np.array(y)

In [ ]:
torch_x = torch.from_numpy(X)
torch_y = torch.from_numpy(y)
y_test = torch.tensor(torch_y,  dtype = torch.long)
x_test = torch.tensor(torch_x , dtype=torch.float32)
#loading fully trained AE
torch.save(autoencoder, 'autoencoder.pkl')

In [ ]:
#feed overservational data and generated data
view_data = Variable(x_test.type(torch.FloatTensor)/255.)
encoded_data, _ = autoencoder(view_data)
fig = plt.figure(1); ax = Axes3D(fig)
X, Y, Z = encoded_data.data[:, 0].numpy(), encoded_data.data[:, 1].numpy(), encoded_data.data[:, 2].numpy()
values = y_test.numpy()
for x, y, z, s in zip(X, Y, Z, values):
    c = cm.rainbow(int(255*s/2)); ax.text(x, y, z, s, backgroundcolor=c)
ax.set_xlim(X.min(), X.max()); ax.set_ylim(Y.min(), Y.max()); ax.set_zlim(Z.min(), Z.max())
plt.savefig("encoded_S4.tif",dpi = 400)
plt.show()